![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [22]:
# QuantBook Analysis Tool
qb = QuantBook()

# Ajouter les paires de cryptomonnaies
crypto_symbols = [
    qb.AddCrypto("BTCUSD").Symbol,
    qb.AddCrypto("ETHUSD").Symbol,
    qb.AddCrypto("LTCUSD").Symbol,
]

# Charger l'historique complet sur la période du backtesting
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 1, 1)
history = qb.History(crypto_symbols, start_date, end_date, Resolution.Hour)

# Vérifier la structure des données
print("Structure des données d'historique :")
print(history.head())

# Partie 1 : Création des indicateurs pour chaque paire
indicators = {}
for symbol in crypto_symbols:
    # Obtenir l'historique pour le symbole
    symbol_history = history.loc[symbol]
    symbol_history = symbol_history[['close', 'high', 'low', 'open', 'volume']]
    symbol_history.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    
    # Vérifier les colonnes
    print(f"Colonnes de {symbol.Value}: {symbol_history.columns}")
    
    # Calculer les indicateurs et stocker dans un dictionnaire
    indicators[symbol] = {
        "rsi": qb.RSI(symbol, 14, Resolution.Hour),
        "ema10": qb.EMA(symbol, 10, Resolution.Hour),
        "ema20": qb.EMA(symbol, 20, Resolution.Hour),
        "ema64": qb.EMA(symbol, 64, Resolution.Hour),
        "ema150": qb.EMA(symbol, 150, Resolution.Hour),
    }

# Limiter les graphiques à une période réduite (1 mois)
plot_start_date = datetime(2023, 1, 1)
plot_end_date = datetime(2023, 2, 1)

# Partie 2 : Simulation des signaux d'achat/vente et visualisation
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

for symbol in crypto_symbols:
    symbol_history = history.loc[symbol]
    symbol_history = symbol_history[['close', 'high', 'low', 'open', 'volume']]
    symbol_history.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    symbol_history = symbol_history[(symbol_history.index >= plot_start_date) & (symbol_history.index <= plot_end_date)]

    # Extraire les indicateurs EMA et RSI
    ema10_df = qb.Indicator(indicators[symbol]["ema10"], symbol, len(history.loc[symbol]), Resolution.Hour)
    ema20_df = qb.Indicator(indicators[symbol]["ema20"], symbol, len(history.loc[symbol]), Resolution.Hour)
    ema64_df = qb.Indicator(indicators[symbol]["ema64"], symbol, len(history.loc[symbol]), Resolution.Hour)
    ema150_df = qb.Indicator(indicators[symbol]["ema150"], symbol, len(history.loc[symbol]), Resolution.Hour)
    rsi_df = qb.Indicator(indicators[symbol]["rsi"], symbol, len(history.loc[symbol]), Resolution.Hour)

    # Filtrer les EMA et RSI pour la période choisie
    ema10_df = ema10_df[(ema10_df.index >= plot_start_date) & (ema10_df.index <= plot_end_date)]
    ema20_df = ema20_df[(ema20_df.index >= plot_start_date) & (ema20_df.index <= plot_end_date)]
    ema64_df = ema64_df[(ema64_df.index >= plot_start_date) & (ema64_df.index <= plot_end_date)]
    ema150_df = ema150_df[(ema150_df.index >= plot_start_date) & (ema150_df.index <= plot_end_date)]
    rsi_df = rsi_df[(rsi_df.index >= plot_start_date) & (rsi_df.index <= plot_end_date)]

    # Simulation des signaux d'achat/vente
    buy_signals = []
    sell_signals = []
    for i, (date, row) in enumerate(ema10_df.iterrows()):
        try:
            price = symbol_history.loc[date]['Close']
            rsi_value = rsi_df.loc[date]['current']
            ema10 = row['current']
            ema20 = ema20_df.loc[date]['current']
            ema64 = ema64_df.loc[date]['current']
            ema150 = ema150_df.loc[date]['current']
            
            # Conditions d'achat
            if rsi_value > 40 and ema10 > ema20 > ema64 > ema150:
                buy_signals.append((date, price))
            
            # Conditions de vente
            if rsi_value < 50 and (ema10 < ema64 or ema20 < ema64):
                sell_signals.append((date, price))
        except KeyError:
            continue

    # Tracer le graphique avec tous les indicateurs
    plt.figure(figsize=(14, 8))
    plt.plot(symbol_history.index, symbol_history['Close'], label=f"{symbol.Value} - Close Price", color='black', linewidth=1.5)
    if not ema10_df.empty:
        plt.plot(ema10_df.index, ema10_df['current'], label="EMA10", linestyle='--', color='blue')
    if not ema20_df.empty:
        plt.plot(ema20_df.index, ema20_df['current'], label="EMA20", linestyle='--', color='green')
    if not ema64_df.empty:
        plt.plot(ema64_df.index, ema64_df['current'], label="EMA64", linestyle='--', color='orange')
    if not ema150_df.empty:
        plt.plot(ema150_df.index, ema150_df['current'], label="EMA150", linestyle='--', color='red')

    # Ajouter les points d'achat/vente
    if buy_signals:
        dates, prices = zip(*buy_signals)
        plt.scatter(dates, prices, marker='^', color='green', label="Buy Signal", s=100)
    if sell_signals:
        dates, prices = zip(*sell_signals)
        plt.scatter(dates, prices, marker='v', color='red', label="Sell Signal", s=100)

    plt.title(f"{symbol.Value} - Close Price, Indicators, and Trading Signals (1 Month)")
    plt.legend()
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid()
    plt.show()

# Partie 3 : Graphique RSI pour chaque symbole
for symbol in crypto_symbols:
    rsi_df = qb.Indicator(indicators[symbol]["rsi"], symbol, len(history.loc[symbol]), Resolution.Hour)
    rsi_df = rsi_df[(rsi_df.index >= plot_start_date) & (rsi_df.index <= plot_end_date)]
    
    plt.figure(figsize=(14, 5))
    plt.plot(rsi_df.index, rsi_df['current'], label=f"{symbol.Value} - RSI", color='purple', linewidth=1.5)
    plt.axhline(50, color='orange', linestyle='--', label="RSI Sell Threshold (50)")
    plt.axhline(40, color='green', linestyle='--', label="RSI Buy Threshold (40)")
    plt.title(f"{symbol.Value} - Relative Strength Index (RSI) (1 Month)")
    plt.legend()
    plt.xlabel("Date")
    plt.ylabel("RSI")
    plt.grid()
    plt.show()

# Partie 4 : Résumé des paires analysées
print(f"Analyse complète pour les paires : {[symbol.Value for symbol in crypto_symbols]}")